<a href="https://colab.research.google.com/github/liicuba/gz-dbt-repository/blob/main/neo_bank_transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#installing google-cloud-bigquery libraries
!pip install google-cloud-bigquery

In [20]:
#Import the library bigquery
from google.cloud import bigquery

#Creates a BigQuery client
client = bigquery.Client()

In [21]:
#google colab auth.
from google.colab import auth
#import pandas library
import pandas as pd

#authenticate the user to access Google Cloud resources
auth.authenticate_user()

# Define the SQL query to retrieve all data from the 'devices' table and specify the project ID
query = "SELECT * FROM `neo-bank-432422.neo_bank.transactions`"
project = "neo-bank-432422"

# Use pandas to execute the SQL query and load the results into a Pandas DataFrame
# The `project_id` argument specifies the Google Cloud project
df_transactions = pd.read_gbq(query=query, project_id=project)

In [22]:
df_transactions.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_17369,EXCHANGE,AED,1.03,COMPLETED,None,NaN,None,None,OUTBOUND,user_6009,2019-05-03 07:19:57.665869+00:00
1,transaction_1539,EXCHANGE,AED,0.02,COMPLETED,None,NaN,None,None,OUTBOUND,user_3300,2018-06-11 12:17:03.139923+00:00
2,transaction_1868,EXCHANGE,AED,3.25,COMPLETED,None,NaN,None,None,OUTBOUND,user_6894,2018-09-27 01:42:40.987191+00:00
3,transaction_1948,EXCHANGE,AED,0.21,COMPLETED,None,NaN,None,None,OUTBOUND,user_397,2018-05-20 12:46:34.103037+00:00
4,transaction_1755821,EXCHANGE,AED,17.55,COMPLETED,None,NaN,None,None,OUTBOUND,user_11513,2018-09-14 04:15:10.192609+00:00


In [23]:
#describe method
df_transactions.describe()

,amount_usd,ea_merchant_mcc
count,2.740075e+06,1.581417e+06
mean,1.703225e+05,5.699736e+03
std,9.634014e+07,9.493660e+02
min,0.000000e+00,7.420000e+02
25%,2.030000e+00,5.411000e+03
50%,8.510000e+00,5.812000e+03
75%,2.822000e+01,5.921000e+03
max,8.510645e+10,9.406000e+03


In [24]:
df_transactions.dtypes

,0
transaction_id,object
transactions_type,object
transactions_currency,object
amount_usd,float64
transactions_state,object
ea_cardholderpresence,object
ea_merchant_mcc,float64
ea_merchant_city,object
ea_merchant_country,object
direction,object


In [25]:
# Check for missing values
#print(df_transactions.isnull().sum() / len(df_transactions)) * 100
#it's more than 30%

In [26]:

# Fill null values in specified columns
df_transactions['ea_cardholderpresence'].fillna('not identified', inplace=True)
df_transactions['ea_merchant_city'].fillna('not identified', inplace=True)
df_transactions['ea_merchant_country'].fillna('not identified', inplace=True)
df_transactions['ea_merchant_mcc'].fillna(0, inplace=True)

df_transactions.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_17369,EXCHANGE,AED,1.03,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6009,2019-05-03 07:19:57.665869+00:00
1,transaction_1539,EXCHANGE,AED,0.02,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_3300,2018-06-11 12:17:03.139923+00:00
2,transaction_1868,EXCHANGE,AED,3.25,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6894,2018-09-27 01:42:40.987191+00:00
3,transaction_1948,EXCHANGE,AED,0.21,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_397,2018-05-20 12:46:34.103037+00:00
4,transaction_1755821,EXCHANGE,AED,17.55,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_11513,2018-09-14 04:15:10.192609+00:00


In [27]:
#double check - nulls
print(df_transactions.isnull().sum())


transaction_id           0
transactions_type        0
transactions_currency    0
amount_usd               0
transactions_state       0
ea_cardholderpresence    0
ea_merchant_mcc          0
ea_merchant_city         0
ea_merchant_country      0
direction                0
user_id                  0
created_date             0
dtype: int64


In [28]:
# Check for duplicates
print("Number of duplicates:",df_transactions.duplicated().sum())

Number of duplicates: 0


Merging tables transactions and users_clean to get the user's country and city


In [29]:
##adding the users_clean table

#authenticate the user to access Google Cloud resources
auth.authenticate_user()

# Define the SQL query to retrieve all data from the 'devices' table and specify the project ID
query = "SELECT * FROM `neo-bank-432422.neo_bank.users_clean`"
project = "neo-bank-432422"

# Use pandas to execute the SQL query and load the results into a Pandas DataFrame
# The `project_id` argument specifies the Google Cloud project
df_users_clean = pd.read_gbq(query=query, project_id=project)

In [32]:
df_users_clean.head()

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,idade
0,user_320,1940,NO,Fyllingsdalen,2018-01-25,0,STANDARD,0.0,0.0,1,0,0,84
1,user_16157,1941,JE,St Helier Jersey,2018-11-29,0,STANDARD,0.0,0.0,0,0,0,83
2,user_1957,1941,HR,Zagreb,2018-02-04,0,STANDARD,1.0,1.0,0,0,0,83
3,user_16541,1944,AT,Wien,2018-11-30,0,STANDARD,0.0,0.0,9,0,0,80
4,user_11097,1945,NO,Bodø,2018-08-19,0,STANDARD,1.0,1.0,0,0,0,79


In [34]:

# Fazer um merge apenas com as colunas desejadas
merged_df = pd.merge(df_transactions, df_users_clean[['user_id', 'country', 'city']], on='user_id', how='left')

# Verify the merge
merged_df.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date,country,city
0,transaction_17369,EXCHANGE,AED,1.03,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6009,2019-05-03 07:19:57.665869+00:00,FR,Saverne
1,transaction_1539,EXCHANGE,AED,0.02,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_3300,2018-06-11 12:17:03.139923+00:00,PL,Gdańsk
2,transaction_1868,EXCHANGE,AED,3.25,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6894,2018-09-27 01:42:40.987191+00:00,LT,Siauliai
3,transaction_1948,EXCHANGE,AED,0.21,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_397,2018-05-20 12:46:34.103037+00:00,HU,Budapest
4,transaction_1755821,EXCHANGE,AED,17.55,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_11513,2018-09-14 04:15:10.192609+00:00,GB,Rickmansworth


In [36]:
#double check - nulls
print(merged_df.isnull().sum())


transaction_id           0
transactions_type        0
transactions_currency    0
amount_usd               0
transactions_state       0
ea_cardholderpresence    0
ea_merchant_mcc          0
ea_merchant_city         0
ea_merchant_country      0
direction                0
user_id                  0
created_date             0
country                  0
city                     0
dtype: int64


In [37]:
# Check for duplicates
print("Number of duplicates:",merged_df.duplicated().sum())

Number of duplicates: 0


In [38]:
merged_df.dtypes

,0
transaction_id,object
transactions_type,object
transactions_currency,object
amount_usd,float64
transactions_state,object
ea_cardholderpresence,object
ea_merchant_mcc,float64
ea_merchant_city,object
ea_merchant_country,object
direction,object


In [42]:
#changing the created_date's type
merged_df['created_date'] = pd.to_datetime(merged_df['created_date']).dt.round('S')

merged_df.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date,country,city
0,transaction_17369,EXCHANGE,AED,1.03,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6009,2019-05-03 07:19:58+00:00,FR,Saverne
1,transaction_1539,EXCHANGE,AED,0.02,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_3300,2018-06-11 12:17:03+00:00,PL,Gdańsk
2,transaction_1868,EXCHANGE,AED,3.25,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6894,2018-09-27 01:42:41+00:00,LT,Siauliai
3,transaction_1948,EXCHANGE,AED,0.21,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_397,2018-05-20 12:46:34+00:00,HU,Budapest
4,transaction_1755821,EXCHANGE,AED,17.55,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_11513,2018-09-14 04:15:10+00:00,GB,Rickmansworth


In [44]:
#populating a table already created in BQ
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("transaction_id", "STRING"),
        bigquery.SchemaField("transactions_type", "STRING"),
        bigquery.SchemaField("transactions_currency", "STRING"),
        bigquery.SchemaField("amount_usd", "FLOAT64"),
        bigquery.SchemaField("transactions_state", "STRING"),
        bigquery.SchemaField("ea_cardholderpresence", "STRING"),
        bigquery.SchemaField("ea_merchant_mcc", "FLOAT64"),
        bigquery.SchemaField("ea_merchant_city", "STRING"),
        bigquery.SchemaField("ea_merchant_country", "STRING"),
        bigquery.SchemaField("direction", "STRING"),
        bigquery.SchemaField("user_id", "STRING"),
        bigquery.SchemaField("created_date", "DATETIME"),
        bigquery.SchemaField("country", "STRING"),
        bigquery.SchemaField("city", "STRING"),

    ],
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Opcional: Truncar (resetar?) a tabela existente
)

client = bigquery.Client(project='neo-bank-432422')

table_ref = client.dataset('neo_bank').table('transactions_clean')
load_job = client.load_table_from_dataframe(merged_df, table_ref, job_config=job_config)
load_job.result()

LoadJob<project=neo-bank-432422, location=EU, id=6f65ee6e-1f51-44ff-86a9-b139ac47302a>

In [45]:
# Save the DataFrame as a CSV file
merged_df.to_csv('transactions_clean.csv', index=False)

#r: Indicates that the file will be opened for reading.
#b: Specifies that the file will be opened in binary mode.

merged_df.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date,country,city
0,transaction_17369,EXCHANGE,AED,1.03,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6009,2019-05-03 07:19:58+00:00,FR,Saverne
1,transaction_1539,EXCHANGE,AED,0.02,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_3300,2018-06-11 12:17:03+00:00,PL,Gdańsk
2,transaction_1868,EXCHANGE,AED,3.25,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_6894,2018-09-27 01:42:41+00:00,LT,Siauliai
3,transaction_1948,EXCHANGE,AED,0.21,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_397,2018-05-20 12:46:34+00:00,HU,Budapest
4,transaction_1755821,EXCHANGE,AED,17.55,COMPLETED,not identified,0.0,not identified,not identified,OUTBOUND,user_11513,2018-09-14 04:15:10+00:00,GB,Rickmansworth
